In [1]:
import glob
import subprocess
import numpy as np

from obspy.core import Stream
from obspy import read, read_inventory, read_events


# from geographiclib.geodesic import Geodesic
from obspy.geodetics.base import gps2dist_azimuth
from obspy.geodetics import kilometers2degrees
# import obspy
from obspy.signal.rotate import rotate_ne_rt

import time

In [2]:
MseedDataDirectory = "../Data/fetch_fdsn_lizhi"
ResponseDirectory = "../Data/fetch_fdsn_sxd/response"

StationCatalog = dict()
with open("../Data/fetch_fdsn_sxd/station.meta", 'r') as f:
    for line in f:
        StationName = line.split('|')[0] + '.' + line.split('|')[1]
        StationCatalog[StationName] = dict()
        StationCatalog[StationName]['NetworkCode'] = line.split('|')[0]
        StationCatalog[StationName]['StationCode'] = line.split('|')[1]
        StationCatalog[StationName]['Channel'] = line.split('|')[3]
        StationCatalog[StationName]['Latitude'] = float(line.split('|')[4])
        StationCatalog[StationName]['Longitude'] = float(line.split('|')[5])

In [3]:
for MseedFileName in glob.glob(MseedDataDirectory+'/*.mseed')[0:1]:
    MseedFileName = glob.glob(MseedDataDirectory+'/*.mseed')[0]
    DataStream = read(MseedFileName,format='MSEED')
    EVENTNAME = MseedFileName.split('/')[-1].split('.')[0]
    print(EVENTNAME)

    CatFind = glob.glob(f"../Data/CMTSOLUTION/{EVENTNAME[0:12]}*.CMTSOLUTION")
    if len(CatFind)==1:
        cat = read_events(CatFind[0])
    else:
        print("CMTSOLUTION not found!!!")
    SourceLat = cat[0].origins[0].latitude
    SourceLon = cat[0].origins[0].longitude
    SourceDepth = cat[0].origins[0].depth/1.0e3

20110113164314


In [4]:
# trim DataStream for same length
DataStream.trim(starttime=DataStream[0].stats.starttime, endtime=DataStream[0].stats.starttime + 2400)
# calculate baz for rotating
for itrace, trace in enumerate(DataStream.select(component="N")):
    StationName = DataStream[itrace].stats.network + "." + DataStream[itrace].stats.station
    distm, azimuth, backazimuth = gps2dist_azimuth(SourceLat,SourceLon,StationCatalog[StationName]['Latitude'],StationCatalog[StationName]['Longitude'])
    distance_in_degrees = kilometers2degrees(distm/1.0e3)
    
    trace.stats.distance = distm
    trace.stats.distance_in_degrees = distance_in_degrees
    trace.stats.azimuth = azimuth
    trace.stats.backazimuth = backazimuth

## Rotate seismogram

In [6]:
RTZDataStream = Stream()

pre_filt = [0.005, 0.01, 5, 10]

for itrace, trace in enumerate(DataStream.select(component="N")):

    TChannelName = trace.stats.network + "." + trace.stats.station + "." \
                 + trace.stats.location + "." "BHT"

    seisCheck = RTZDataStream.select(id=TChannelName)
    if len(seisCheck)>0:
        print('processed trace exists!!!')
        continue

    print(trace)
    EChannelName = trace.stats.network + "." + trace.stats.station + "." \
                 + trace.stats.location + "." "BHE"
    seisE = DataStream.select(id=EChannelName)
    
    NChannelName = trace.stats.network + "." + trace.stats.station + "." \
                 + trace.stats.location + "." "BHN"
    seisN = DataStream.select(id=NChannelName)

    ZChannelName = trace.stats.network + "." + trace.stats.station + "." \
                 + trace.stats.location + "." "BHZ"
    seisZ = DataStream.select(id=ZChannelName)

    if len(seisE) != 1 or len(seisN) != 1:
        # Merge segaments into one single trace
        seisE.merge(method=1,fill_value='interpolate',interpolation_samples=2)
        seisN.merge(method=1,fill_value='interpolate',interpolation_samples=2)
        seisZ.merge(method=1,fill_value='interpolate',interpolation_samples=2)
    

    # Remove response
    try:
        invE = read_inventory(ResponseDirectory+"/RESP."+EChannelName)
        seisE.remove_response(inventory=invE, pre_filt=pre_filt, output="DISP") 
        invN = read_inventory(ResponseDirectory+"/RESP."+NChannelName)
        seisN.remove_response(inventory=invN, pre_filt=pre_filt, output="DISP")
        invZ = read_inventory(ResponseDirectory+"/RESP."+ZChannelName)
        seisZ.remove_response(inventory=invZ, pre_filt=pre_filt, output="DISP")
    except:
        print(trace, "remove response failed")
        continue

    if len(seisN[0].data) != len(seisE[0].data):
        seisN.resample(100)
        seisE.resample(100)
        trimstart = max(seisN[0].stats.starttime, seisE[0].stats.starttime)
        trimend = min(seisN[0].stats.endtime, seisE[0].stats.endtime)
        seisN.trim(trimstart, trimend)
        seisE.trim(trimstart, trimend)
    # Rotate from NE to RT component
    [seisRtmp,seisTtmp] = rotate_ne_rt(seisN[0].data, seisE[0].data, seisN[0].stats.backazimuth)
    
    seisR=seisN[0].copy()
    seisR.stats['channel']='BHR'
    seisR.data=seisRtmp
    seisT=seisN[0].copy()
    seisT.stats['channel']='BHT'
    seisT.data=seisTtmp
    
    RTZDataStream += seisR
    RTZDataStream += seisT
    RTZDataStream += seisZ[0]

AH.ANQ.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.BAS.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.BEB.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.CHZ.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.DYN.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.FZL.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.HBE.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.HEF.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.HNA.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
AH.HSH.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 24000

/home/zhi/anaconda3/lib/python3.8/site-packages/obspy/core/inventory/network.py:251: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


BJ.ZHT.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
CQ.WAZ.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples


CQ.CHK.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples


CQ.CHS.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples


...

 EVRESP ERROR (... [File: <stdin>; Start date: ; Stage: 2]):
	check_channel; units mismatch between stages,
	skipping to next response now


XJ.XNY.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
XJ.XNY.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples remove response failed
XJ.YCH.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples


 EVRESP ERROR (... [File: <stdin>; Start date: ; Stage: 2]):
	check_channel; units mismatch between stages,
	skipping to next response now


XJ.YJS.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:08:45.990000Z | 100.0 Hz, 183200 samples
processed trace exists!!!
processed trace exists!!!
XJ.YMS.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
XJ.YMS.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples remove response failed
XJ.YUT.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples
XJ.YUT.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples remove response failed
XJ.ZSU.00.BHN | 2011-01-13T16:38:14.000000Z - 2011-01-13T17:18:14.000000Z | 100.0 Hz, 240001 samples


 EVRESP ERROR (... [File: <stdin>; Start date: ; Stage: 2]):
	check_channel; units mismatch between stages,
	skipping to next response now
 EVRESP ERROR (... [File: <stdin>; Start date: ; Stage: 2]):
	check_channel; units mismatch between stages,
	skipping to next response now


In [6]:
RTZDataStream.resample(10)
RTZDataStream.write(f"../Data/ProcessedSecondRequest/{EVENTNAME}.PICKLE",format='PICKLE')
print(f"../Data/ProcessedSecondRequest/{EVENTNAME}.PICKLE Saved!!!")

../Data/ProcessedSecondRequest/20110113164314.PICKLE Saved!!!
